In [ ]:
from google.colab import drive

# Force remount to refresh credentials
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/Datasets/DIV2K_motion_blurred/"

blurred  kernels  sharp


In [ ]:
import numpy as np
import os

KERNEL_DIR = "/content/drive/MyDrive/Datasets/DIV2K_defocus_blurred/kernels"
bad_files = []
for fname in os.listdir(KERNEL_DIR):
    if fname.endswith(".npy"):
        arr = np.load(os.path.join(KERNEL_DIR, fname))
        if arr.shape != (15, 15):
            bad_files.append((fname, arr.shape))

print(f"Talált hibás PSF fájlok: {len(bad_files)}")
print(bad_files[:5])  # csak az első 5 hibás fájl kiírása

Talált hibás PSF fájlok: 800
[('0001.npy', (5, 5)), ('0002.npy', (7, 7)), ('0003.npy', (11, 11)), ('0004.npy', (11, 11)), ('0005.npy', (11, 11))]


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from tqdm import tqdm

# --- Paraméterek ---
DATA_DIR = "/content/drive/MyDrive/Datasets/DIV2K_defocus_blurred"
KERNEL_SIZE = 15
BATCH_SIZE = 16
EPOCHS = 20
LR = 1e-4

# --- Dataset definíció ---
class PSFDataset(Dataset):
    def __init__(self, blur_dir, kernel_dir, transform=None):
        self.blur_dir = blur_dir
        self.kernel_dir = kernel_dir
        self.filenames = sorted([f for f in os.listdir(blur_dir) if f.endswith('.png')])
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        fname = self.filenames[idx]
        blur_path = os.path.join(self.blur_dir, fname)
        kernel_path = os.path.join(self.kernel_dir, fname.replace('.png', '.npy'))

        image = Image.open(blur_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        kernel = np.load(kernel_path)
        kernel = torch.from_numpy(kernel).float().unsqueeze(0)  # [1, 15, 15]

        return image, kernel

# --- Transzformáció ---
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# --- Dataset & DataLoader ---
dataset = PSFDataset(
    blur_dir=os.path.join(DATA_DIR, "blurred"),
    kernel_dir=os.path.join(DATA_DIR, "kernels"),
    transform=transform
)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# --- Modell definíció ---
class PSFPredictor(nn.Module):
    def __init__(self, kernel_size=15):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(64, kernel_size * kernel_size)
        self.kernel_size = kernel_size

    def forward(self, x):
        x = self.encoder(x)  # [B, 64, 1, 1]
        x = x.view(x.size(0), -1)  # [B, 64]
        x = self.fc(x)             # [B, 225]
        x = x.view(-1, 1, self.kernel_size, self.kernel_size)
        x = F.relu(x)              # non-negatív
        x = x / (x.sum(dim=[2, 3], keepdim=True) + 1e-8)  # normalizálás
        return x

# --- Tanítás ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PSFPredictor(kernel_size=KERNEL_SIZE).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for imgs, kernels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, kernels = imgs.to(device), kernels.to(device)
        pred = model(imgs)
        loss = criterion(pred, kernels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"✅ Epoch {epoch+1}: Loss = {total_loss / len(dataloader):.6f}")

# --- Modell mentése ---
os.makedirs("/content/drive/MyDrive/models", exist_ok=True)
torch.save(model.state_dict(), "/content/drive/MyDrive/models/psf_predictor_defocus.pth")
print("\n📅 Modell mentve: psf_predictor_defocus.pth")

Epoch 1/20: 100%|██████████| 50/50 [03:01<00:00,  3.62s/it]


✅ Epoch 1: Loss = 0.000039


Epoch 2/20: 100%|██████████| 50/50 [00:16<00:00,  3.07it/s]


✅ Epoch 2: Loss = 0.000025


Epoch 3/20: 100%|██████████| 50/50 [00:16<00:00,  2.99it/s]


...

✅ Epoch 18: Loss = 0.000011


Epoch 19/20: 100%|██████████| 50/50 [00:17<00:00,  2.84it/s]


✅ Epoch 19: Loss = 0.000011


Epoch 20/20: 100%|██████████| 50/50 [00:16<00:00,  2.96it/s]

✅ Epoch 20: Loss = 0.000011

📅 Modell mentve: psf_predictor_defocus.pth
